In [25]:
#import modules
from EDA import Preprocessing
import pandas as pd
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')

from sklearn.compose import make_column_selector as selector
from sklearn.preprocessing import OrdinalEncoder
from sklearn.model_selection import train_test_split
from IPython.display import display


In [ ]:
#read datasets
import pandas as pd

df_mat = pd.read_csv('/home/pirate/Documents/machine_learning/score_prediction/student/student-mat.csv',sep=';')
df_por = pd.read_csv('/home/pirate/Documents/machine_learning/score_prediction/student/student-por.csv',sep=';')


In [ ]:
#Handling duplicates found in datasets
df_merged=df_mat.append(df_por, ignore_index=True)
no_of_duplicates = df_merged.duplicated(["school","sex","age","address","famsize","Pstatus","Medu","Fedu","Mjob","Fjob","reason","nursery","internet"]).sum()
print(f'---> We have {no_of_duplicates} found in our two datasets\n')

print('---> Merging datasets and removing duplicates\n')
df_merged.drop_duplicates(subset=["school","sex","age","address","famsize","Pstatus","Medu","Fedu","Mjob","Fjob","reason","nursery","internet"],
                 keep = 'first', inplace = True)

print(f'---> After merging and removing duplicates we have {len(df_merged)} unique rows')


In [ ]:
                    # PERFORMING EDA
preprocessor = Preprocessing(df_merged)
#check for missing values
df = preprocessor.check_missing_value()

#check dataset descriptives
preprocessor.descriptives(df)

#perform data exploration
##separating dataset into categorical and continuous columns
numerical_columns_selector = selector(dtype_exclude=object)
categorical_columns_selector = selector(dtype_include=object)
continous_columns = numerical_columns_selector(df)
categorical_columns = categorical_columns_selector(df)


In [ ]:
colors = sns.color_palette('pastel')[0:5]

#performing counts of catgorical variable levels for some variables
# sex_count_table = pd.crosstab(index=df['sex'], columns='freq')
sns.countplot(x ='sex', data = df)
plt.show()

sns.boxplot(x ='guardian', y ='studytime', data = df, hue ='sex')
plt.show()

sns.stripplot(x ='school', y ='G3', data = df, jitter = True, hue ='sex', dodge = True)
plt.show()

# sns.pairplot(df, hue ="sex", palette ='coolwarm')
# plt.show()

g = sns.catplot(x='guardian',y='studytime', col = 'romantic', data=df,
                kind='bar', aspect=.6, palette='Set2')
(g.set_axis_labels("Guardian", "Study Time")
  .set_titles("relationship : {col_name}"))
plt.show()

sns.barplot(x='internet',y='G3',data=df, palette='rainbow', hue='sex')
plt.show()


In [22]:
#DATASET ENCODING OF CATEGORICAL VARIABLES
df_copy = df.copy() #copying the dataset to preserve original copy
ord_enc = OrdinalEncoder() #encoder object

#looping through all the categorical columns initailly found and applying encoding on it
print('Performing EnCoding Of Categorical variables')
for col in categorical_columns:
    df_copy[col] = ord_enc.fit_transform(df_copy[[col]])
df_copy.head(10)

,school,sex,age,address,famsize,Pstatus,Medu,Fedu,Mjob,Fjob,...,freetime,goout,Dalc,Walc,health,absences,G1,G2,G3,Mjob_code
0,0.0,0.0,18,1.0,0.0,0.0,4,4,0.0,4.0,...,3,4,1,1,3,6,5,6,6,0.0
1,0.0,0.0,17,1.0,0.0,1.0,1,1,0.0,2.0,...,3,3,1,1,3,4,5,5,6,0.0
2,0.0,0.0,15,1.0,1.0,1.0,1,1,0.0,2.0,...,3,2,2,3,3,10,7,8,10,0.0
3,0.0,0.0,15,1.0,0.0,1.0,4,2,1.0,3.0,...,2,2,1,1,5,2,15,14,15,1.0
4,0.0,0.0,16,1.0,0.0,1.0,3,3,2.0,2.0,...,3,2,1,2,5,4,6,10,10,2.0
5,0.0,1.0,16,1.0,1.0,1.0,4,3,3.0,2.0,...,4,2,1,2,5,10,15,15,15,3.0
6,0.0,1.0,16,1.0,1.0,1.0,2,2,2.0,2.0,...,4,4,1,1,3,0,12,12,11,2.0
7,0.0,0.0,17,1.0,0.0,0.0,4,4,2.0,4.0,...,1,4,1,1,1,6,6,5,6,2.0
8,0.0,1.0,15,1.0,1.0,0.0,3,2,3.0,2.0,...,2,2,1,1,1,0,16,18,19,3.0
9,0.0,1.0,15,1.0,0.0,1.0,3,4,2.0,2.0,...,5,1,1,1,5,0,14,15,15,2.0


In [24]:
                        #REGRESSION
#splitting datasets into dependent and independent variables
X = df_copy.drop('G3',axis=1)
Y = df_copy['G3']

#splitting X, Y into train and test (80:20) ratio
x_train,x_test,y_train,y_test=train_test_split(X,Y,test_size=0.2)

0     6
1     6
2    10
3    15
4    10
Name: G3, dtype: int64